In [28]:
import pandas as pd
import geopandas as gpd
import numpy as np
import mysql.connector
from mysql.connector import Error
from joblib import Parallel, delayed
import replicaEVSE.sql_wrapper_functions as sql
import replicaEVSE.load_curve_simulation_functions as sim
import dask.dataframe as dd
from dask.distributed import Client
%reload_ext autoreload
%autoreload 1

In [29]:

client = Client()
client

/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 55370 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:55370/status,
Dashboard: http://127.0.0.1:55370/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:55371,Workers: 4
Dashboard: http://127.0.0.1:55370/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:55383,Total threads: 2
Dashboard: http://127.0.0.1:55390/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:55374,


In [5]:
! ls ../../data

EIA_demand_summary.csv                  northwest_2021_Q4_saturday_trip.csv
Untitled.ipynb                          northwest_2021_Q4_saturday_trip.parquet
blockgroup_boundaries.pkl               northwest_2021_Q4_thursday_trip.csv
blockgroups_53.zip                      northwest_2021_Q4_thursday_trip.parquet
northwest_2021_Q4_population.csv        test.csv
northwest_2021_Q4_population.parquet


In [30]:
datadir = '../../data'

In [31]:
pd.set_option('display.max_columns', None)

In [32]:
with open('../src/metadata/MySQLpwd.txt') as f:
    pw = f.readlines()
pw = pw[0]

connection_replica = sql.create_db_connection("localhost", "root", pw, 'replica')

MySQL Database connection successful


Read in blockgroup boundary geodataframe that defines simulation area

In [10]:
gdf = pd.read_pickle(datadir+'/blockgroup_boundaries.pkl')
# gdf

Create list of blockgroup ids from which to pull data for the simulation

In [11]:
bgrp_list = list(gdf.GEOID.values)

In [10]:
def pull_person_list(connection, blockgroups):
    """Pulls list of person_ids from SQL database given a list of destination blockgroups.

    Blockgroups are smaller than census tracts, so this is a more accurate representation of the population in the area.
    
    Parameters
    ----------
    connection : MySQL connector
    blockgroups : list
        List of destination blockgroups. The person_id attached to any trip ending in the blockgroup will be used as part of the simulation population
        
    Returns
    -------
    list
        List of person_ids used in simulation
    """
    
    q1 = """
    SELECT person_id
    FROM trips
    WHERE destination_bgrp IN """+str(tuple(blockgroups))+""";
    """
    results_people = sql.read_query(connection, q1)
    persons_df = sql.convert_to_df(results_people,['person_id'])
    persons_df = persons_df.drop_duplicates()
    return(list(set(persons_df.person_id)))

In [12]:
blockgroups = bgrp_list
q1 = """
    SELECT person_id
    FROM trips
    WHERE destination_bgrp IN """+str(tuple(blockgroups))+""";
    """

results_people = sql.read_query(connection_replica, q1)


In [66]:
q1

"\n    SELECT person_id\n    FROM trips\n    WHERE destination_bgrp IN ('530299714002', '530299714003', '530299715001', '530299715002', '530299716001', '530330033001', '530530729031', '530530730015', '530530730013', '530530730014', '530299716002', '530299718001', '530299718002', '530299719001', '530299719002', '530299719003', '530299701002', '530299701001', '530299720001', '530299720002', '530299720003', '530299721002', '530299721003', '530610528032', '530610528033', '530610528034', '530610528042', '530610528043', '530610528044', '530610528052', '530299702001', '530299703003', '530299704002', '530299704003', '530299705001', '530299705002', '530299705003', '530299706011', '530299706012', '530610528053', '530610528051', '530610528062', '530610528063', '530299707001', '530299707002', '530299708001', '530299708002', '530299709001', '530299709002', '530299709003', '530299710001', '530299706013', '530299706021', '530299706023', '530330323113', '530610528064', '530610529063', '530610529034', 

In [26]:
top = """
SELECT *
FROM population
LIMIT 10;"""
sql.read_query(connection_replica, top)


[('13869309696222166396',
  '10000000323569557755',
  '410192100004',
  '410190500023',
  'nan',
  '41019210000',
  '41019050002',
  'nan',
  '35_64',
  46,
  'F',
  'white',
  'not_hispanic_or_latino',
  '0_20000',
  887.0,
  'employed',
  'high_school',
  'not_attending_school',
  'naics541940',
  'child',
  '1',
  'husband/wife_no_children',
  'driving',
  'owner',
  'same_house',
  '3_person',
  '40000_75000',
  70097.0,
  'family_single',
  '2',
  'single_family',
  'core',
  'english',
  42.790351685227975,
  -123.26420573748364,
  43.38126246542247,
  -123.31829046103536,
  None,
  None,
  'worked_from_home'),
 ('9351892862469751559',
  '10000001964381556008',
  '160199705023',
  '160199714002',
  '160199706022',
  '16019970502',
  '16019971400',
  '16019970602',
  '18_24',
  22,
  'M',
  'white',
  'not_hispanic_or_latino',
  '20000_40000',
  21468.0,
  'employed',
  'some_college',
  'undergraduate',
  'naics44_45',
  'child',
  '0',
  'nan',
  'driving',
  'owner',
  'same_ho

In [18]:
people_list = list(set(results_people))

In [16]:
person_list = np.unique(results_people)

In [17]:
person_list

array(['10000002909730821040', '10000003183049770926',
       '10000005160936665771', ..., '9999992107991804948',
       '9999996696186290995', '9999997956849808929'], dtype='<U20')

In [ ]:
results_people = sql.read_query(connection_replica, q1)


In [29]:
len(results_people)/1e6

51.727268

Pull person list to create simulation population

In [35]:
trip_sat = dd.read_parquet(datadir+'/northwest_2021_Q4_saturday_trip.parquet')
trip_thu = dd.read_parquet(datadir+'/northwest_2021_Q4_thursday_trip.parquet')
trips = dd.concat([trip_sat, trip_thu])
people = trips['person_id'].loc[trips['destination_bgrp'].isin(blockgroups)].drop_duplicates().compute()

2023-04-17 16:06:38,094 - distributed.worker.memory - WARNING - Worker is at 80% memory usage. Pausing worker.  Process memory: 3.24 GiB -- Worker memory limit: 4.00 GiB
2023-04-17 16:06:39,267 - distributed.worker.memory - WARNING - Worker is at 79% memory usage. Resuming worker. Process memory: 3.20 GiB -- Worker memory limit: 4.00 GiB


In [37]:
chunked_list = np.array_split(people.values, 1000)

In [28]:
#Number of people in the simulation
len(people)/1e6

51.727268

In [38]:
chunked_list[0]

array(['9261044664253989731', '11705321199965034957',
       '12047641940475444253', ..., '5011435391917840520',
       '14470373545714533185', '4751800203279003086'], dtype=object)

Define colums to pull for simulation and read in existing load to use for charging optimization

In [40]:
person_columns = [
    'household_id', 'person_id', 'BLOCKGROUP', 'BLOCKGROUP_work', 'age',
    'sex', 'race', 'ethnicity', 'individual_income', 'employment', 'education',
    'industry', 'household_role', 'commute_mode', 'household_size', 'household_income',
    'family_structure', 'vehicles', 'building_type', 'resident_type'
]
trips_columns = [
    'activity_id', 'person_id', 'distance_miles', 'mode', 'travel_purpose', 'start_time',
    'end_time', 'origin_bgrp', 'destination_bgrp', 'origin_land_use_l1', 'destination_land_use_l1',
    'origin_building_use_l1', 'destination_building_use_l1', 'vehicle_type','weekday'
]
existing_load=dd.read_csv('../../data/EIA_demand_summary.csv') #Created in the EIA_data_download.ipynb notebook

In [34]:
def create_lists(chunk_size, person_list):
    """Creates list of person_id lists of a given chunk_size for parallelizing simulation
    Parameters
    ----------
    chunk_size : int
        Number of person ids to include in each sub-list
        The simulation runs each sub-list of people on a single core in series
    person_list : list
        Person ids to reorganize into list of lists
    Returns
    -------
    list
        List of lists of person ids
    """
    
    lists = [person_list[x:x+chunk_size] for x in range(0,len(person_list),chunk_size)]
    return(lists)

Divide list of person_ids into 100-person sub-lists for parallelization

In [55]:
# chunked_list = create_lists(100,person_list)
client.close()

In [69]:
from dask.distributed import Client, LocalCluster

# replace with whichever cluster class you're using
# https://docs.dask.org/en/stable/deploying.html#distributed-computing
cluster = LocalCluster()
# connect client to your cluster
client = Client(cluster)

# Monitor your computation with the Dask dashboard
print(client.dashboard_link)

http://127.0.0.1:8787/status


In [74]:
type(chunked_list[0])

numpy.ndarray

Run two simulations—one with unmanaged charging, and the other with managed charging

In [77]:
%%time
import time
import joblib
try: 
    with joblib.parallel_backend("dask"):
        joblib.Parallel(verbose=100)(
            joblib.delayed(sim.run_charge_simulation)(
            person_ids = id,
            database_name = 'replica',
            database_pw = pw,
            person_columns = person_columns,
            trips_columns = trips_columns,
            existing_load = existing_load,
            simulation_id = '01',
            managed = False
        ) for id in chunked_list)
except ValueError:
    print('ValueError')
    breakpoint()
    pass

[Parallel(n_jobs=-1)]: Using backend DaskDistributedBackend with 8 concurrent workers.
MySQL Database connection successful
4751800203279003086
MySQL Database connection successful
8311751139682729740
MySQL Database connection successful
14182926702180916408
MySQL Database connection successful
15853988323055856254
MySQL Database connection successful
1099278117153328983
MySQL Database connection successful
17432528149800507545
MySQL Database connection successful
1135797921520336679
MySQL Database connection successful
12485798019439483730
MySQL Database connection successful
11491538083635106173
MySQL Database connection successful
2010179665154515022
MySQL Database connection successful
15190527745866557710
ValueError
CPU times: user 45.7 s, sys: 5 s, total: 50.7 s
Wall time: 4min 16s
MySQL Database connection successful
3714597675295938247


2023-04-14 16:09:42,583 - distributed.worker - WARNING - Compute Failed
Key:       batch_of_run_charge_simulation_1_calls-0397d74432834781b39ea6adb1f465b9
Function:  execute_task
args:      ((<function apply at 0x101abefc0>, batch_of_run_charge_simulation_1_calls, [], {'tasks': [(<function run_charge_simulation at 0x11031f060>, [], {'person_ids': array(['9261044664253989731', '11705321199965034957',
       '12047641940475444253', ..., '5011435391917840520',
       '14470373545714533185', '4751800203279003086'], dtype=object), 'database_name': 'replica', 'database_pw': 'I8anapple!', 'person_columns': ['household_id', 'person_id', 'BLOCKGROUP', 'BLOCKGROUP_work', 'age', 'sex', 'race', 'ethnicity', 'individual_income', 'employment', 'education', 'industry', 'household_role', 'commute_mode', 'household_size', 'household_income', 'family_structure', 'vehicles', 'building_type', 'resident_type'], 'trips_columns': ['activity_id', 'person_id', 'distance_miles', 'mode', 'travel_purpose', 'start

MySQL Database connection successful
15586902494152624785
MySQL Database connection successful
1194056039050078940


2023-04-14 16:09:42,658 - distributed.worker - WARNING - Compute Failed
Key:       batch_of_run_charge_simulation_1_calls-51b271666ec94982ae1986e15dc62bfb
Function:  execute_task
args:      ((<function apply at 0x101b86fc0>, batch_of_run_charge_simulation_1_calls, [], {'tasks': [(<function run_charge_simulation at 0x121b0efc0>, [], {'person_ids': array(['10591755769114010983', '1445514123502694080',
       '3504683318827351069', ..., '12084875375661462272',
       '9642967758337622468', '1135797921520336679'], dtype=object), 'database_name': 'replica', 'database_pw': 'I8anapple!', 'person_columns': ['household_id', 'person_id', 'BLOCKGROUP', 'BLOCKGROUP_work', 'age', 'sex', 'race', 'ethnicity', 'individual_income', 'employment', 'education', 'industry', 'household_role', 'commute_mode', 'household_size', 'household_income', 'family_structure', 'vehicles', 'building_type', 'resident_type'], 'trips_columns': ['activity_id', 'person_id', 'distance_miles', 'mode', 'travel_purpose', 'start_

MySQL Database connection successful
9548148166314694287
MySQL Database connection successful
11919653896776563605


2023-04-14 16:13:14,369 - distributed.worker - WARNING - Compute Failed
Key:       batch_of_run_charge_simulation_1_calls-c7cfda74a139486db25067da8c31b7f4
Function:  execute_task
args:      ((<function apply at 0x101abefc0>, batch_of_run_charge_simulation_1_calls, [], {'tasks': [(<function run_charge_simulation at 0x11031f060>, [], {'person_ids': array(['9929388252069785530', '17269564605726224992',
       '7596365437332440583', ..., '6441355836116851125',
       '3847671475764732765', '11491538083635106173'], dtype=object), 'database_name': 'replica', 'database_pw': 'I8anapple!', 'person_columns': ['household_id', 'person_id', 'BLOCKGROUP', 'BLOCKGROUP_work', 'age', 'sex', 'race', 'ethnicity', 'individual_income', 'employment', 'education', 'industry', 'household_role', 'commute_mode', 'household_size', 'household_income', 'family_structure', 'vehicles', 'building_type', 'resident_type'], 'trips_columns': ['activity_id', 'person_id', 'distance_miles', 'mode', 'travel_purpose', 'start_

In [ ]:
temp = Parallel(n_jobs=6)(
    delayed(sim.run_charge_simulation)(
        person_ids = i,
        database_name = 'replica',
        database_pw = pw,
        person_columns = person_columns,
        trips_columns = trips_columns,
        existing_load = existing_load,
        simulation_id = '02',
        managed = True
    ) for i in chunked_list)#

MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
MySQL Database connection successful
M

In [51]:
type(chunked_list[0])

numpy.ndarray

In [54]:
people

12         9261044664253989731
446       11705321199965034957
447       12047641940475444253
932       16409299978104629971
933       13147923906478375057
                  ...         
180668    16593036485312744089
180886    14578818647173159764
181059     4714525973063744216
181138    10147352607296064792
181298      103932989148128297
Name: person_id, Length: 8538399, dtype: object

In [53]:
people.loc[people['person_id'] == '10236253046352761306']

KeyError: 'person_id'

In [52]:


sim.run_charge_simulation(
            person_ids = chunked_list[0][0:100],
            database_name = 'replica',
            database_pw = pw,
            person_columns = person_columns,
            trips_columns = trips_columns,
            existing_load = existing_load,
            simulation_id = '01',
            managed = False
        )

MySQL Database connection successful
11088826869089647247


ValueError: Cannot call len() on object with unknown chunk size.

A possible solution: https://docs.dask.org/en/latest/array-chunks.html#unknown-chunks
Summary: to compute chunks sizes, use

   x.compute_chunk_sizes()  # for Dask Array `x`
   ddf.to_dask_array(lengths=True)  # for Dask DataFrame `ddf`

In [ ]:
#Use this in case you want to delete data from the simulation database to rerun the simulation
# q = """
# DELETE FROM charges;
# """
# tests = sql.execute_query(connection_replica, q)

In [79]:
trip_sat['travel_purpose'].value_counts().compute()

HOME                   15798932
SHOP                   13008868
EAT                     6415163
SOCIAL                  4990435
RECREATION              2576654
WORK                    2497691
MAINTENANCE             2059801
COMMERCIAL              1267956
OTHER_ACTIVITY_TYPE      826816
LODGING                  455022
SCHOOL                   228930
REGION_DEPARTURE          86868
WORK_FROM_HOME            84327
Name: travel_purpose, dtype: int64

In [80]:
trip_sat['mode'].value_counts().compute()

PRIVATE_AUTO         24585018
CARPOOL              18456496
WALKING               4035579
COMMERCIAL            1258639
OTHER_TRAVEL_MODE      910208
ON_DEMAND_AUTO         460807
BIKING                 341998
PUBLIC_TRANSIT         248718
Name: mode, dtype: int64

In [81]:
trip_sat.loc[trip_sat['mode'] == 'COMMERCIAL'].compute()


,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
0,9279000034943401070,17327509686180360634,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 15:23:12,0 days 16:35:35,62.733511,MEDIUM_COMMERCIAL,060150001011,41.746259,-124.206273,410159502003,42.562838,-124.337297,nan,nan,nan,nan,nan,nan,nan,nan,41.74826,-124.20112,42.50703,-124.40986
2,1647238923281467213,15293783543053158940,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:24:07,0 days 17:28:07,54.388781,HEAVY_COMMERCIAL,060490002001,41.735255,-121.352275,410359720002,42.259899,-121.817733,nan,nan,nan,nan,nan,nan,nan,nan,41.68983,-121.27189,42.25545,-121.79924
3,14161940105906798253,10230136483887035220,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 09:55:16,0 days 13:16:49,208.460835,MEDIUM_COMMERCIAL,060490002001,41.735255,-121.352275,410191500001,43.151013,-123.373297,nan,nan,nan,nan,nan,nan,nan,nan,41.76158,-121.30210,43.15260,-123.37303
4,11456520016899244980,11447961505550466002,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 14:05:25,0 days 15:52:41,111.543443,HEAVY_COMMERCIAL,060490002001,41.735255,-121.352275,410359701003,43.285156,-121.558591,nan,nan,nan,nan,nan,nan,nan,nan,41.76055,-121.30210,43.12554,-121.80610
6,1557433381049270022,559983492497788695,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:33:24,0 days 16:41:34,4.132399,MEDIUM_COMMERCIAL,060930001001,41.811117,-121.600244,060930001001,41.811117,-121.600244,nan,nan,nan,nan,nan,nan,nan,nan,41.99777,-121.51084,41.95387,-121.48612
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85878,12910140291689604387,867206370902095125,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:05:34,0 days 16:36:26,32.300392,MEDIUM_COMMERCIAL,560430003021,44.019316,-107.939480,560179678001,43.648248,-108.215819,nan,nan,nan,nan,nan,nan,nan,nan,44.01701,-107.96607,43.64750,-108.20915
85879,3510582306494789933,12813755273707020993,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 06:45:39,0 days 06:56:45,9.916767,MEDIUM_COMMERCIAL,560430003021,44.019316,-107.939480,560430002001,44.007320,-108.234660,nan,nan,nan,nan,nan,nan,nan,nan,44.03676,-107.95097,43.92707,-108.05534
85880,5901084494108477005,345241192582764009,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 10:36:08,0 days 16:45:59,416.060081,HEAVY_COMMERCIAL,560430003021,44.019316,-107.939480,300770001001,46.997671,-113.049478,nan,nan,nan,nan,nan,nan,nan,nan,44.02392,-107.96470,46.60746,-113.01979
85881,16449000756089704285,9386307840054704208,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 10:09:15,0 days 10:27:15,19.572888,MEDIUM_COMMERCIAL,560459511001,43.989531,-104.796388,560119502003,44.272116,-104.951558,nan,nan,nan,nan,nan,nan,nan,nan,44.10188,-104.63310,44.26339,-104.95170


In [83]:
mhd_people = trip_sat['person_id'].loc[trip_sat['mode'] == 'COMMERCIAL'].drop_duplicates().compute()

In [105]:
trucker_trips_ids = trip_sat.loc[trip_sat['person_id'].isin(mhd_people.values)]

In [113]:
trucker_trips_ids['origin_land_use_l2'].value_counts(dropna=False).compute()

nan    1258639
Name: origin_land_use_l2, dtype: int64

In [122]:
id = "17327509686180360634"
trucker_trips_ids.loc[trucker_trips_ids['person_id'] == id].compute()

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
0,9279000034943401070,17327509686180360634,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 15:23:12,0 days 16:35:35,62.733511,MEDIUM_COMMERCIAL,060150001011,41.746259,-124.206273,410159502003,42.562838,-124.337297,nan,nan,nan,nan,nan,nan,nan,nan,41.74826,-124.20112,42.50703,-124.40986


In [108]:
trucker_trips_ids.head(50)

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
0,9279000034943401070,17327509686180360634,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 15:23:12,0 days 16:35:35,62.733511,MEDIUM_COMMERCIAL,060150001011,41.746259,-124.206273,410159502003,42.562838,-124.337297,nan,nan,nan,nan,nan,nan,nan,nan,41.74826,-124.20112,42.50703,-124.40986
2,1647238923281467213,15293783543053158940,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:24:07,0 days 17:28:07,54.388781,HEAVY_COMMERCIAL,060490002001,41.735255,-121.352275,410359720002,42.259899,-121.817733,nan,nan,nan,nan,nan,nan,nan,nan,41.68983,-121.27189,42.25545,-121.79924
3,14161940105906798253,10230136483887035220,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 09:55:16,0 days 13:16:49,208.460835,MEDIUM_COMMERCIAL,060490002001,41.735255,-121.352275,410191500001,43.151013,-123.373297,nan,nan,nan,nan,nan,nan,nan,nan,41.76158,-121.30210,43.15260,-123.37303
4,11456520016899244980,11447961505550466002,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 14:05:25,0 days 15:52:41,111.543443,HEAVY_COMMERCIAL,060490002001,41.735255,-121.352275,410359701003,43.285156,-121.558591,nan,nan,nan,nan,nan,nan,nan,nan,41.76055,-121.30210,43.12554,-121.80610
6,1557433381049270022,559983492497788695,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 16:33:24,0 days 16:41:34,4.132399,MEDIUM_COMMERCIAL,060930001001,41.811117,-121.600244,060930001001,41.811117,-121.600244,nan,nan,nan,nan,nan,nan,nan,nan,41.99777,-121.51084,41.95387,-121.48612
7,3880256203351854608,2251190701348025175,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 18:56:26,0 days 20:46:26,114.651978,MEDIUM_COMMERCIAL,060930002001,41.822763,-121.972017,410359701002,43.532480,-121.481571,nan,nan,nan,nan,nan,nan,nan,nan,41.96306,-121.91871,43.52515,-121.64268
8,701406414104553361,4703855435004264260,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 14:34:20,0 days 15:26:35,29.522208,MEDIUM_COMMERCIAL,060930002001,41.822763,-121.972017,410359707001,42.060713,-121.750772,nan,nan,nan,nan,nan,nan,nan,nan,41.86598,-121.96403,42.02838,-121.61384
9,8493762862756967127,15106259212856995599,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 11:49:02,0 days 12:38:17,49.731732,MEDIUM_COMMERCIAL,060930003003,41.638474,-122.255252,410359720003,42.268736,-121.790533,nan,nan,nan,nan,nan,nan,nan,nan,41.64367,-122.13569,42.25647,-121.79924
10,2029980988630214698,3869566259793052094,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 22:42:07,0 days 00:29:23,115.659919,MEDIUM_COMMERCIAL,060930004001,41.917386,-122.644732,410191700001,43.032329,-123.413924,nan,nan,nan,nan,nan,nan,nan,nan,41.99675,-122.60948,43.06236,-123.34693
11,13620739091214446578,17380067703204696292,COMMERCIAL,COMMERCIAL,COMMERCIAL,0 days 18:09:21,0 days 19:07:02,60.720618,MEDIUM_COMMERCIAL,060930007013,41.622580,-122.622828,410290010022,42.378391,-122.915811,nan,nan,nan,nan,nan,nan,nan,nan,41.66624,-122.60948,42.37630,-122.91709


In [125]:
trucker_trips_ids['person_id'].dtype, pop['person_id'].dtype

(dtype('O'), dtype('O'))

In [86]:
mhd = """
    SELECT *
    FROM population
    WHERE destination_bgrp IN """+str(tuple(mhd_people.values))+""";
    """

results_people = sql.read_query(connection_replica, q1)


'../../data'

In [97]:
! ls ../../data/

88448.75s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
EIA_demand_summary.csv                  northwest_2021_Q4_saturday_trip.csv
Untitled.ipynb                          northwest_2021_Q4_saturday_trip.parquet
blockgroup_boundaries.pkl               northwest_2021_Q4_thursday_trip.csv
blockgroups_53.zip                      northwest_2021_Q4_thursday_trip.parquet
northwest_2021_Q4_population.csv        test.csv
northwest_2021_Q4_population.parquet


In [100]:
pop = dd.read_parquet('../../data/northwest_2021_Q4_population.parquet')
truckers = pop.loc[

In [101]:
truckers.head()

/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/dask/dataframe/core.py:7985: UserWarning: Insufficient elements for `head`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh


In [99]:
pop.head()

,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh
0,7071162995671019379,11747437169840333628,410390044034,nan,nan,41039004403,nan,nan,65_plus,75,F,white,not_hispanic_or_latino,0_20000,12760.0,unemployed,some_college,not_attending_school,not_working,GQ,0,nan,not_working,GQ,within_us,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,44.033699,-123.118654,NaN,NaN,NaN,NaN,unemployed
1,8269392924797733378,1756119472378340340,530670102003,530670101004,nan,53067010200,53067010100,nan,35_64,50,M,white,not_hispanic_or_latino,0_20000,5050.0,employed,k_12,not_attending_school,naics56,GQ,0,nan,transit,GQ,same_house,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,47.051348,-122.876651,47.031766,-122.896907,NaN,NaN,worked_from_home
2,16986606175343716434,298012200302628452,560210004021,560210002003,nan,56021000402,56021000200,nan,18_24,19,M,white,not_hispanic_or_latino,0_20000,10314.0,employed,some_college,not_attending_school,not_working,GQ,0,nan,driving,GQ,same_house,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,41.119804,-104.751563,41.123491,-104.817224,NaN,NaN,worked_in_person
3,18413256466912027068,11331589210799163497,530150021001,530150021001,nan,53015002100,53015002100,nan,18_24,20,F,white,hispanic_or_latino,100000_plus,233381.0,employed,some_college,not_attending_school,not_working,GQ,0,nan,walking,GQ,within_us,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,46.128480,-122.936175,46.128287,-122.935595,NaN,NaN,worked_in_person
4,5180439401743391780,8335433671007283715,530330053022,530330053022,nan,53033005302,53033005302,nan,18_24,20,F,white,not_hispanic_or_latino,0_20000,10734.0,employed,some_college,undergraduate,naics611310,GQ,0,nan,walking,GQ,within_us,1_person_group_quarters,lte_10000,0.0,GQ,GQ,GQ_structure,core,GQ_language,47.648779,-122.308424,47.652771,-122.313792,NaN,NaN,worked_in_person


In [127]:
mhd_people.values

array(['17327509686180360634', '15293783543053158940',
       '10230136483887035220', ..., '345241192582764009',
       '9386307840054704208', '12908899829320780928'], dtype=object)

In [95]:
pop.head()

/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/dask/dataframe/core.py:7985: UserWarning: Insufficient elements for `head`. 5 elements requested, only 0 elements available. Try passing larger `npartitions` to `head`.
  warnings.warn(


,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh


In [129]:
trip = trip_sat.sort_values(by='person_id')
pop = pop.sort_values(by='person_id')

In [130]:
trip.head()

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
81452,1961671362429628690,10000001964381556008,PRIVATE_AUTO,SCHOOL,HOME,0 days 07:03:12,0 days 07:12:19,3.211177,nan,160199705023,43.474461,-111.954449,160199706022,43.489489,-111.995056,residential,single_family,residential,single_family,civic_institutional,education,civic_institutional,education,43.471088,-111.952067,43.487031,-111.985908
18521,131863418729036517,10000001964381556008,WALKING,SHOP,HOME,0 days 17:53:00,0 days 18:08:22,0.796125,nan,160199705023,43.474461,-111.954449,160199705031,43.441962,-111.973130,residential,single_family,residential,single_family,commercial,retail,commercial,retail,43.471088,-111.952067,43.467211,-111.964696
113989,16471291535434355943,10000001964381556008,PRIVATE_AUTO,HOME,SHOP,0 days 18:20:22,0 days 18:23:41,0.788874,nan,160199705031,43.441962,-111.973130,160199705023,43.474461,-111.954449,commercial,retail,commercial,retail,residential,single_family,residential,single_family,43.467211,-111.964696,43.471088,-111.952067
129072,2492297005633067743,10000001964381556008,PRIVATE_AUTO,HOME,SCHOOL,0 days 16:59:17,0 days 17:09:19,3.211177,nan,160199706022,43.489489,-111.995056,160199705023,43.474461,-111.954449,civic_institutional,education,civic_institutional,education,residential,single_family,residential,single_family,43.487031,-111.985908,43.471088,-111.952067
87863,10652465939061186874,10000002909730821040,PRIVATE_AUTO,HOME,WORK,0 days 21:04:21,0 days 21:38:18,9.284612,nan,530250110001,47.131321,-119.203190,530250111002,47.098457,-119.319023,commercial,retail,commercial,retail,residential,single_family,residential,single_family,47.132725,-119.213000,47.085370,-119.319090


In [131]:
pop.head()

,household_id,person_id,BLOCKGROUP,BLOCKGROUP_work,BLOCKGROUP_school,TRACT,TRACT_work,TRACT_school,age_group,age,sex,race,ethnicity,individual_income_group,individual_income,employment,education,school_grade_attending,industry,household_role,subfamily_number,subfamily_relationship,commute_mode,tenure,migration,household_size,household_income_group,household_income,family_structure,vehicles,building_type,resident_type,language,lat,lng,lat_work,lng_work,lat_school,lng_school,wfh
94313,13869309696222166396,10000000323569557755,410192100004,410190500023,nan,41019210000,41019050002,nan,35_64,46,F,white,not_hispanic_or_latino,0_20000,887.0,employed,high_school,not_attending_school,naics541940,child,1,husband/wife_no_children,driving,owner,same_house,3_person,40000_75000,70097.0,family_single,2,single_family,core,english,42.790352,-123.264206,43.381262,-123.31829,NaN,NaN,worked_from_home
145045,9351892862469751559,10000001964381556008,160199705023,160199714002,160199706022,16019970502,16019971400,16019970602,18_24,22,M,white,not_hispanic_or_latino,20000_40000,21468.0,employed,some_college,undergraduate,naics44_45,child,0,nan,driving,owner,same_house,3_person,125000_plus,134181.0,married_couple,3_plus,single_family,core,english,43.471088,-111.952067,NaN,NaN,43.487031,-111.985908,worked_in_person
151148,17571687911180552742,10000002909730821040,530250111002,530250110001,nan,53025011100,53025011000,nan,35_64,64,M,white,hispanic_or_latino,100000_plus,103145.0,employed,some_college,not_attending_school,naics53,spouse,0,nan,driving,owner,same_house,2_person,125000_plus,211447.0,married_couple,3_plus,single_family,core,english,47.085370,-119.319090,47.132725,-119.21300,NaN,NaN,worked_in_person
133697,17212045348023710843,10000003183049770926,530110408032,nan,nan,53011040803,nan,nan,65_plus,68,F,white,not_hispanic_or_latino,20000_40000,32310.0,notinlf,some_college,not_attending_school,not_working,head_of_household,0,nan,not_working,owner,same_house,2_person,75000_125000,104124.0,married_couple,2,single_family,core,english,45.702720,-122.632958,NaN,NaN,NaN,NaN,under_16_or_notinlf
42428,6736242328113975455,10000003900118609948,410670314031,nan,nan,41067031403,nan,nan,65_plus,78,M,white,not_hispanic_or_latino,80000_100000,81152.0,notinlf,some_college,not_attending_school,not_working,head_of_household,0,nan,not_working,owner,same_house,3_person,75000_125000,114483.0,married_couple,3_plus,single_family,core,english,45.514461,-122.806445,NaN,NaN,NaN,NaN,under_16_or_notinlf


In [ ]:
pop.loc[pop['person_id'] == '10000001964381556008'].compute()

2023-04-14 17:44:07,387 - distributed.shuffle - ERROR - shuffle_transfer failed during shuffle 6e163875b3fca3c262a953aa1d89e011
Traceback (most recent call last):
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/tornado/iostream.py", line 973, in _handle_write
    num_bytes = self.write_to_fd(self._write_buffer.peek(size))
                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/tornado/iostream.py", line 1146, in write_to_fd
    return self.socket.send(data)  # type: ignore
           ^^^^^^^^^^^^^^^^^^^^^^
OSError: [Errno 55] No buffer space available

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/shuffle/_shuffle.py", line 58, in shuffle_transfer
    return _get_worker_extension().add_partition(
           ^^^^^^^^^^^^

RuntimeError: shuffle_transfer failed during shuffle 6e163875b3fca3c262a953aa1d89e011

2023-04-14 17:44:07,974 - distributed.core - ERROR - Exception while handling op shuffle_get
Traceback (most recent call last):
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/core.py", line 828, in _handle_comm
    result = handler(**msg)
             ^^^^^^^^^^^^^^
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/shuffle/_scheduler_extension.py", line 127, in get
    state = self.states[id]
            ~~~~~~~~~~~^^^^
KeyError: '6e163875b3fca3c262a953aa1d89e011'


2023-04-14 17:44:08,007 - distributed.core - ERROR - Exception while handling op shuffle_receive
Traceback (most recent call last):
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/core.py", line 830, in _handle_comm
    result = await result
             ^^^^^^^^^^^^
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/shuffle/_worker_extension.py", line 587, in shuffle_receive
    shuffle = await self._get_shuffle_run(shuffle_id, run_id)
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/shuffle/_worker_extension.py", line 678, in _get_shuffle_run
    shuffle = await self._refresh_shuffle(
              ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/matthew.wilde/mambaforge/envs/py311/lib/python3.11/site-packages/distributed/shuffle/_worker_extension.py", line 749, in _refresh_shuffle
    result = await s

In [59]:
problem = trips.loc[trips['person_id'] == '17832631922058752069'].compute()

2023-04-17 17:05:32,482 - distributed.scheduler - WARNING - Worker failed to heartbeat within 300 seconds. Closing: <WorkerState 'tcp://127.0.0.1:49848', name: 1, status: running, memory: 0, processing: 3>
2023-04-17 17:05:50,788 - distributed.nanny - WARNING - Worker process still alive after 3.1999960327148442 seconds, killing


In [60]:
problem

,activity_id,person_id,mode,travel_purpose,previous_activity_type,start_time,end_time,distance_miles,vehicle_type,origin_bgrp,origin_bgrp_lat,origin_bgrp_lng,destination_bgrp,destination_bgrp_lat,destination_bgrp_lng,origin_land_use_l1,origin_land_use_l2,origin_building_use_l1,origin_building_use_l2,destination_land_use_l1,destination_land_use_l2,destination_building_use_l1,destination_building_use_l2,origin_lat,origin_lng,destination_lat,destination_lng
108498,16874096940537531455,17832631922058752069,WALKING,SOCIAL,LODGING,0 days 07:30:00,0 days 07:30:00,0.000000,nan,530330284032,47.439883,-122.291595,530330284032,47.439883,-122.291595,commercial,retail,commercial,retail,commercial,retail,commercial,retail,47.437312,-122.287622,47.437289,-122.287595
114882,3611042350494732001,17832631922058752069,PRIVATE_AUTO,SHOP,SOCIAL,0 days 07:42:00,0 days 07:49:55,3.439509,nan,530330284032,47.439883,-122.291595,530330262001,47.464988,-122.233845,commercial,retail,commercial,retail,commercial,office,commercial,retail,47.437289,-122.287595,47.458555,-122.258554
106624,12277342496296167424,17832631922058752069,PRIVATE_AUTO,SHOP,SHOP,0 days 09:30:00,0 days 09:37:33,2.557958,nan,530330262001,47.464988,-122.233845,530330262001,47.464988,-122.233845,commercial,office,commercial,retail,commercial,retail,commercial,retail,47.458555,-122.258554,47.471326,-122.218900
66941,15189215599697005260,17832631922058752069,PRIVATE_AUTO,MAINTENANCE,LODGING,0 days 10:26:00,0 days 10:34:56,3.156956,nan,530330284032,47.439883,-122.291595,530330262001,47.464988,-122.233845,commercial,retail,commercial,retail,commercial,office,commercial,office,47.437312,-122.287622,47.456247,-122.259304
60561,17175012580755845915,17832631922058752069,CARPOOL,LODGING,MAINTENANCE,0 days 12:23:00,0 days 12:35:32,5.381641,nan,530330262001,47.464988,-122.233845,530330258051,47.448808,-122.191489,commercial,office,commercial,office,mixed_use,office,commercial,office,47.456247,-122.259304,47.449476,-122.189093
36214,15444420399981297726,17832631922058752069,PRIVATE_AUTO,LODGING,SHOP,0 days 16:01:00,0 days 16:11:15,3.991730,nan,530330262001,47.464988,-122.233845,530330293053,47.426786,-122.205197,commercial,retail,commercial,retail,residential,single_family,residential,single_family,47.471326,-122.218900,47.427611,-122.199303
115338,1366743695637767250,17832631922058752069,PRIVATE_AUTO,SHOP,LODGING,0 days 17:58:00,0 days 18:06:10,2.904485,nan,530330258051,47.448808,-122.191489,530330262001,47.464988,-122.233845,mixed_use,office,commercial,office,commercial,retail,commercial,office,47.449476,-122.189093,47.471709,-122.220236
73325,2831652192349944436,17832631922058752069,PRIVATE_AUTO,LODGING,SHOP,0 days 20:33:00,0 days 20:42:09,4.358946,nan,530330262001,47.464988,-122.233845,530330293053,47.426786,-122.205197,commercial,retail,commercial,office,residential,single_family,residential,single_family,47.471709,-122.220236,47.426960,-122.204309


In [62]:
problem['person_id'].dtype

dtype('O')

In [64]:
pop = dd.read_parquet('../../data/northwest_2021_Q4_population.parquet')

dtype('O')